# Bayesian Cosmological Parameter Estimation with Gravitational-Wave Standard Sirens

This notebook demonstrates **Bayesian inference in a high-dimensional cosmological model**
using **synthetic gravitational-wave (GW) standard siren data**.

We fit the parameters:
- $H_0$ (Hubble constant)
- $\Omega_m$ (matter density)
- $\Omega_\Lambda$ (dark energy density)
- $\Omega_r$ (radiation density)
- $w_0, w_a$ (dark energy equation of state: CPL parameterization)
- $T_{\rm CMB}$ (CMB temperature)

This example illustrates why **Bayesian methods are essential** for modern, multi-parameter cosmology.

## 1. Cosmological Model

We use a generalized Friedmann equation:

$$H(z)^2 = H_0^2 \left[ \Omega_m(1+z)^3 + \Omega_r(1+z)^4 + \Omega_\Lambda f_{\rm DE}(z) \right]$$

with CPL dark energy:

$$w(z) = w_0 + w_a \frac{z}{1+z}$$

$$f_{\rm DE}(z) = (1+z)^{3(1+w_0+w_a)} \exp\left(-3w_a \frac{z}{1+z}\right)$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

c = 3e5  # km/s

def E(z, H0, Om, Ode, Or, w0, wa):
    fde = (1+z)**(3*(1+w0+wa)) * np.exp(-3*wa*z/(1+z))
    return np.sqrt(Om*(1+z)**3 + Or*(1+z)**4 + Ode*fde)

def luminosity_distance(z, params):
    H0, Om, Ode, Or, w0, wa, Tcmb = params
    zgrid = np.linspace(0, z, 300)
    integrand = c / (H0 * E(zgrid, H0, Om, Ode, Or, w0, wa))
    return (1+z) * np.trapz(integrand, zgrid)

## 2. Synthetic Gravitational-Wave Standard Siren Data

We generate mock GW events with:
- Measured luminosity distance
- Identified host galaxy redshift

Distance uncertainties are large, reflecting realistic GW observations.

In [ ]:
np.random.seed(1)

# True cosmological parameters
params_true = [70, 0.3, 0.7, 9e-5, -1.0, 0.0, 2.725]

z = np.sort(np.random.uniform(0.01, 1.5, 30))
dL_true = np.array([luminosity_distance(zi, params_true) for zi in z])

sigma_dL = 0.15 * dL_true  # 15% GW distance errors
dL_obs = dL_true + np.random.normal(0, sigma_dL)

plt.errorbar(z, dL_obs, yerr=sigma_dL, fmt='o', label='GW sirens')
plt.plot(z, dL_true, label='True cosmology')
plt.xlabel('Redshift z')
plt.ylabel('Luminosity Distance (Mpc)')
plt.legend()
plt.show()

## 3. Likelihood Function

Assuming Gaussian distance errors:

$$\ln \mathcal{L} = -\frac{1}{2}\sum \frac{(d_L^{\rm obs} - d_L^{\rm model})^2}{\sigma_{d_L}^2}$$

In [ ]:
def log_likelihood(params):
    H0, Om, Ode, Or, w0, wa, Tcmb = params
    if Om < 0 or Ode < 0 or Or < 0:
        return -np.inf
    model = np.array([luminosity_distance(zi, params) for zi in z])
    return -0.5 * np.sum((dL_obs - model)**2 / sigma_dL**2)

## 4. Priors

We impose broad, physically motivated priors to stabilize inference.

In [ ]:
def log_prior(params):
    H0, Om, Ode, Or, w0, wa, Tcmb = params
    if not (50 < H0 < 90): return -np.inf
    if not (0 < Om < 1): return -np.inf
    if not (0 < Ode < 1): return -np.inf
    if not (0 < Or < 1e-3): return -np.inf
    if not (-2 < w0 < 0): return -np.inf
    if not (-2 < wa < 2): return -np.inf
    if not (2 < Tcmb < 3): return -np.inf
    return 0.0

def log_posterior(params):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params)

## 5. Simple MCMC Sampler (Metropolis–Hastings)

Grid methods fail in 7D parameter space.
We use a basic MCMC sampler to explore the posterior.

In [ ]:
nsteps = 6000
step_scale = np.array([1, 0.02, 0.02, 1e-5, 0.05, 0.05, 0.01])

chain = np.zeros((nsteps, 7))
chain[0] = params_true + np.random.normal(0, step_scale)

logp = log_posterior(chain[0])

for i in range(1, nsteps):
    proposal = chain[i-1] + np.random.normal(0, step_scale)
    logp_prop = log_posterior(proposal)
    if np.log(np.random.rand()) < logp_prop - logp:
        chain[i] = proposal
        logp = logp_prop
    else:
        chain[i] = chain[i-1]

burn = 1000
samples = chain[burn:]

## 6. Posterior Constraints

We extract marginal constraints from the MCMC chain.

In [ ]:
labels = ['H0','Om','Ode','Or','w0','wa','Tcmb']

for i, lab in enumerate(labels):
    mean = np.mean(samples[:, i])
    std = np.std(samples[:, i])
    print(f'{lab}: {mean:.3f} ± {std:.3f}')

## 7. Interpretation

This example shows:
- Strong degeneracies between cosmological parameters
- The necessity of priors in high-dimensional inference
- Why χ² minimization is insufficient

**Bayesian inference allows us to extract meaning, not just best fits.**